# 工数予測　サンプルコード

当サンプルコードは、”工数予測”のテーマのAI課題において、配布されたファイルを分析し、投稿ファイルを作成するまでの大まかな流れを、ソースコードや実行結果とともに示したものとなっています。詳細な分析を行う足がかりとしてご利用ください。 当サンプルコードは、以下のような流れで構成されています。

1. 前分析
1. モデリング
1. 評価用データに対する予測
1. 応募用結果ファイルの作成

前分析ではデータを読み込んで可視化などを行い、データの規模感や目的変数の性質などを確認します。モデリングでは学習用データを用いて正味作業時間と付帯作業時間を予測するモデルを作成します。評価用データに対する予測では、評価期間に対応する正味作業時間と付帯作業時間を予測して、結果を出力します。応募用結果ファイルの作成では予測した正味作業時間と付帯作業時間をもとに投稿用のファイルを作成します。  

必要なライブラリは以下です。

- pandas
- numpy
- matplotlib
- scikit-learn

使用するデータは以下です。

- actual_train.csv
- processing_train.csv
- base_train.csv
- actual_test.csv
- processing_test.csv

サイトからダウンロードして当ノートブックと同じディレクトリに配置してください。なお、以下のようなフォルダ構成を前提としています。

```
PBL03（Colabの場合は"DXQuest"となっている想定）
│  PBL03_sample_code.ipynb
│  readme.md
│  requirements.txt
│  sample_submit.csv
│
├─test
│      actual_test.csv
│      base_test.csv
│      processing_test.csv
│
└─train
        actual_train.csv
        base_train.csv
        processing_train.csv
```

## 0. 初期設定
- こちらはColabを利用される際の初期設定となります。Colabを利用されない方はこちらの実行は不要です。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- ファイルパスの設定
    - 利用される環境に応じて片方をコメントアウト、片方を実行する形で修正ください。
    - 下記ではGoogle Colaboratoryで実行する想定としています。

In [ ]:
import os

## Google Colabを利用する場合
path = '/content/drive/MyDrive/DXQuest/'

## ローカルの場合
# path = "./"

## 1. 前分析

まずデータの読み込みと可視化を行うために必要なライブラリをインポートします

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### データの読み込みと中身の確認

基本データ、加工データ、実績データを読み込みます。

In [1]:
base_train = pd.read_csv(path + 'train/base_train.csv')
processing_train = pd.read_csv(path + 'train/processing_train.csv')
actual_train = pd.read_csv(path + 'train/actual_train.csv')

NameError: name 'pd' is not defined

それぞれの行数や列数とデータ型などを確認します。

In [ ]:
base_train.info()

In [ ]:
processing_train.info()

In [ ]:
actual_train.info()

それぞれ行数や列数が多く、データ型も混在していて、欠損値も目立つことが分かります。

### 目的変数の分布の確認

目的変数は正味作業時間と付帯作業時間です。これらは実績データの中に存在し、それぞれカラム名は"作業時間"と"合計時間"となります。まずはそれぞれの基礎統計量を確認します。

In [ ]:
actual_train[['作業時間', '合計時間']].describe()

作業時間に0や合計時間に負の値の存在が認められます。

"作業日"を日付データに変換し、2020-02-04以降の"合計時間"が意味を持つ(付帯作業時間に対応)レコードのみ抽出します。  
また、評価対象となる"号機名"はグルアー、2号機、4号機、6号機、7号機、8号機です。それらのレコードのみ抽出します。  
さらに0や負の値などを除外したデータを改めて作成します。

In [1]:
actual_train['作業日'] = pd.to_datetime(actual_train['作業日'])
actual_filtered = actual_train[actual_train['号機名'].isin(['グルアー','2号機','4号機','6号機','7号機','8号機'])]
actual_filtered = actual_filtered[actual_filtered['作業日']>='2020-02-04']
actual_filtered = actual_filtered[(actual_filtered['作業時間']>0)&(actual_filtered['合計時間']>0)]
print(actual_filtered.shape)

NameError: name 'pd' is not defined

データが絞られたのが分かります。

まず正味作業時間の分布を確認してみます。

In [ ]:
plt.hist(actual_filtered['作業時間'], bins=100)
plt.show()

続いて付帯作業時間の分布を確認してみます。

In [ ]:
plt.hist(actual_filtered['合計時間'], bins=100)
plt.show()

両者ともに形は似ていて、大きく左に寄っていることが分かります。

改めて基礎統計量も見てみます。

In [ ]:
actual_filtered[['作業時間','合計時間']].describe()

付帯作業時間（合計時間）のほうが比較的長い傾向にあるようです。もう少しデータをフィルタリングしたらまた違った性質が見えてくるかもしれません。色々なフィルタリングの仕方を考え、データを眺めてみましょう。

## 2. モデリング

続いて、正味作業時間と付帯作業時間を予測するモデルを構築します。

### 学習データの作成

まずはモデルに学習させるデータを作成します。説明変数としては直接作業に影響があると思われる加工データの"数量1"を用いることにします。  
まず、目的変数と説明変数が並ぶように加工データと実績データを統合します。キーは"受注番号"と"号機名"です。  
また、"合計時間"が意味を持つレコードに絞り、関係する変数のみ抜き出します。

In [ ]:
train_merged = pd.merge(processing_train, actual_train, on = ['受注番号','号機名'])
train_merged = train_merged[train_merged['作業日']>='2020-02-04']
train_data = train_merged[['受注番号', '号機名', '作業日','数量1', '作業時間', '合計時間']]

評価対象となる"号機名"はグルアー、2号機、4号機、6号機、7号機、8号機なので、"号機名"はそれらに絞ります。"作業時間"や"合計時間"が0や負のレコードも除外します。

In [ ]:
train_data = train_data[train_data['号機名'].isin(['グルアー','2号機','4号機','6号機','7号機','8号機'])]
train_data = train_data[(train_data['作業時間']>0)&(train_data['合計時間']>0)]
train_data.info()

モデルに学習させるデータは数値である必要があります。  
"数量1"には文字列が含まれる(Dtypeがobjectとなっている)ようなので、全て数値データに変換します。

In [ ]:
train_data['数量1'] = train_data['数量1'].astype(float)
train_data.info()

### モデルの学習

作成した学習データで説明変数が"数量1"で目的変数が正味作業時間と付帯作業時間となるようなモデルを学習させます。今回は線形回帰モデルを用います。

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()

学習データから改めて学習用と検証用のデータを作成します。  
"作業日"が2020-06-01以前を学習用、以降を検証用とします。

In [ ]:
train_all = train_data[train_data['作業日']<'2020-06-01']
val_all = train_data[train_data['作業日']>='2020-06-01']

学習用でモデルを学習させます。

In [ ]:
regr.fit(train_all[['数量1']], train_all[['作業時間', '合計時間']])

### モデルの精度評価

検証用でモデルの精度を確認します。  
今回はMAE(mean absolute error)により正味作業時間("作業時間")と付帯作業時間("合計時間")の精度を評価します。

In [ ]:
import numpy as np

y_hat = regr.predict(val_all[['数量1']])
print('正味作業時間のMAE:', np.abs(val_all['作業時間']-y_hat[:,0]).mean())
print('付帯作業時間のMAE:', np.abs(val_all['合計時間']-y_hat[:,1]).mean())

正味作業時間のMAE: 9.952812099542541
付帯作業時間のMAE: 18.54542886954304


予実プロットを作成してみます。まずは正味作業時間です。

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=90)
ax.set_xlabel('prediction', fontsize=10)
ax.set_ylabel('ground truth', fontsize=10)
ax.set_yscale("linear")
ax.plot([i for i in range(int(val_all['作業時間'].max()))],[i for i in range(int(val_all['作業時間'].max()))], color='red')
ax.scatter(y_hat[:,0], val_all['作業時間'])
ax.grid()
ax.set_aspect('equal')
plt.show()

赤い線に近い点が多ければ多いほど精度が高いことを表します。そこそこ当てられているようです。

続いて付帯作業時間です。

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=90)
ax.set_xlabel('prediction', fontsize=10)
ax.set_ylabel('ground truth', fontsize=10)
ax.set_yscale("linear")
ax.plot([i for i in range(int(val_all['合計時間'].max()))],[i for i in range(int(val_all['合計時間'].max()))], color='red')
ax.scatter(y_hat[:,1], val_all['合計時間'])
ax.grid()
ax.set_aspect('equal')
plt.show()

付帯作業時間は当てるのが難しそうです。

余裕があれば他の機械学習モデルを試してみましょう。

## 3. 評価用データに対する予測

学習したモデルを用いて、評価用データに対して正味作業時間と付帯作業時間を出力します。  
まず作成した学習用データと同様の評価用データを作成します。

In [ ]:
processing_test = pd.read_csv(path + 'test/processing_test.csv')
actual_test = pd.read_csv(path + 'test/actual_test.csv')
test_merged = pd.merge(actual_test, processing_test)
test_data = test_merged[['index','受注番号','号機名','数量1']]
test_data.info()

"数量1"が数値データ(Dtypeがfloat)となっていることが確認できます。  
作成したデータに対して予測を行います。

In [ ]:
y_hat = regr.predict(test_data[['数量1']])

## 4. 応募用結果ファイルの作成

得られた結果を応募用ファイルのフォーマットに合わせます。フォーマットは以下のようにします。  

|index|正味作業時間|付帯作業時間|
|----|----|----|
|0|予測0_0|予測1_0|
|...|...|...|
|5982|予測0_5982|予測1_5982|

In [ ]:
submit = pd.DataFrame({'index':test_data['index'], '正味作業時間':y_hat[:,0], '付帯作業時間':y_hat[:,1]})

ヘッダーはなしなので、含まないように注意します。また、DataFrameが持っているindexも除外する必要があります(4列のデータになってしまうため)。

In [ ]:
submit.to_csv(path + 'my_submission.csv', index=None, header=False)

作成した"my_submission.csv"が最終的な応募用結果ファイルとなります。  
モデリングなどで試行錯誤して繰り返し投稿をして、スコアを最適化しましょう。